### Classificação multiclasse iris com validação cruzada

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 2.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch.nn as nn        
from skorch import NeuralNetClassifier
import torch
from sklearn.model_selection import cross_val_score
torch.__version__

'2.0.0+cu118'

### Base de dados

está sendo definida uma semente (seed) para a geração de números aleatórios nos módulos NumPy (np.random.seed(123)) e PyTorch (torch.manual_seed(123)).

As sementes são usadas para controlar a geração de números pseudoaleatórios. Ao definir uma semente específica, garante-se que a sequência de números gerada seja a mesma em diferentes execuções do código. Isso é útil para fins de reprodutibilidade, ou seja, garantir que os resultados do código sejam consistentes e reproduzíveis.

In [3]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [6]:
base.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [9]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder() #  Essa instância será usada para realizar a codificação numérica.
classe = encoder.fit_transform(classe)

In [12]:
np.unique(classe) 

array([0, 1, 2])

In [13]:
#DataFrame.astype() O método é usado para converter um objeto de pandas em um dtype especificado. astype()
previsores = previsores.astype('float32')
classe = classe.astype('int64')

### Construção do modelo

Essa parte do código define uma classe chamada classificador_torch que herda da classe nn.Module, que é uma classe base do PyTorch para a criação de modelos de aprendizado profundo.

A classe classificador_torch implementa um modelo de classificação utilizando camadas densas (fully connected) e ativações ReLU.

Em resumo, essa classe define um modelo de classificação simples com três camadas densas e duas camadas de ativação ReLU. Durante o treinamento, os dados de entrada serão passados através dessas camadas, produzindo as previsões finais do modelo.

In [18]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(4, 16)
        self.activation0 = nn.ReLU()
        self.dense1 = nn.Linear(16, 16)
        self.activation1 = nn.ReLU()
        self.dense2 = nn.Linear(16, 3)
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dense2(X)
        return X

Essa parte do código cria um classificador utilizando a classe NeuralNetClassifier do scikit-learn, que permite usar modelos PyTorch como base. O classificador é configurado com o modelo classificador_torch, a função de perda CrossEntropyLoss, o otimizador Adam e outros parâmetros relacionados ao treinamento, como o número máximo de épocas e o tamanho do lote.

* module -> Especifica o modelo a serutilizado como base para o classificador.

* criterion -> Define a função de perda a ser utilizada durante o treinamento do classificador.

* optimizer -> Especifica o otimizador a ser utilizado para ajustar os pesos do modelo durante o treinamento.

* max_epochs -> Define o número máximo de épocas durante o treinamento do classificador

* batch_size -> Especifica o tamanho do (lote) batch de dados que vai ser utilizado no treinamento.

* train_split -> Indica se deve ser feita uma divisao automatica entre conjuntos de treinamento e validação durante o treinamento.

In [19]:
from skorch.net import optimizer_setter
classificador_sklearn = NeuralNetClassifier(module = classificador_torch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 1000,
                                            batch_size = 10,
                                            train_split = False)

### Validação cruzada

A função cross_val_score realiza a validação cruzada do classificador, dividindo o conjunto de dados em "folds" de treinamento e teste e retornando uma lista com os resultados de desempenho do classificador para cada fold. No caso desse código, a lista de resultados é armazenada na variável resultados.

In [20]:
resultados = cross_val_score(classificador_sklearn, previsores, classe, cv = 5, 
                             scoring = 'accuracy')

  epoch    train_loss     dur
-------  ------------  ------
      1        2.0070  0.2215
      2        1.0293  0.0182
      3        0.9454  0.0185
      4        0.8550  0.0188
      5        0.7365  0.0173
      6        0.6514  0.0180
      7        0.5581  0.0176
      8        0.5056  0.0188
      9        0.5162  0.0191
     10        0.4109  0.0183
     11        0.5542  0.0188
     12        0.4098  0.0185
     13        0.5332  0.0189
     14        0.4993  0.0185
     15        0.3979  0.0182
     16        0.3369  0.0172
     17        0.3687  0.0205
     18        0.3944  0.0174
     19        0.2903  0.0258
     20        0.3097  0.0184
     21        0.2867  0.0211
     22        0.2556  0.0184
     23        0.2571  0.0198
     24        0.2474  0.0220
     25        0.2309  0.0181
     26        0.2160  0.0202
     27        0.2134  0.0163
     28        0.2090  0.0246
     29        0.2031  0.0268
     30        0.1998  0.0253
     31        0.1923  0.0242
     32   

In [21]:
media = resultados.mean()
desvio = resultados.std()

In [22]:
media, desvio

(0.9733333333333334, 0.038873012632301994)

In [23]:
resultados

array([1.        , 1.        , 0.96666667, 0.9       , 1.        ])